# Fase 2: Hiperafinación y entrenamiento

## Configuración de la GPU

Por un motivo que se desconoce, cuando se utiliza el acelerador P100, es necesario limitar el crecimiento de la GPU (para más detalles, revisar [acá](https://www.tensorflow.org/guide/gpu#limiting_gpu_memory_growth)). Otro punto es que esta celda debe venir antes de cualquier importación ya que internamente modifican la capacidad de la GPU (ver el siguiente [hilo](https://github.com/hunglc007/tensorflow-yolov4-tflite/issues/171)), y por ende, obtenemos el error `Physical devices cannot be modified after being initialized`.

In [1]:
from tensorflow.config import list_physical_devices
from tensorflow.config.experimental import set_memory_growth

gpus = list_physical_devices('GPU')

if gpus:
    try:
        for gpu in gpus:
            set_memory_growth(gpu, True)
    except RuntimeError as e:
        print("error:", e)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

## Instalación de dependencias

Esta imagen viene con librerías preinstaladas de Tensorflow y Hugging Face, por lo que únicamente deberemos instalar `dotwiz`, el cual permite transformar un diccionario a una notación de puntos (tal y como accedemos a métodos y atributos en Javascript). Como haremos modificaciones a los conjuntos de datos, es necesario copiarlos porque están contenidos en el directorio `/kaggle/working` y es de lectura únicamente.

In [2]:
%%capture
!pip install dotwiz
!cp -r /kaggle/input/32bs-24mbs-5e-5lr-4e ./

Cambiaremos el nivel de *logs* generados por Hugging Face ya que genera advertencias que no aplican en nuestro caso.

In [3]:
from transformers import logging

logging.set_verbosity_error()

## Inicialización de constantes

Definiremos al objeto `CONSTANTS` que contendrá todas las constantes a utilizar organizadas de forma jerárquica.

In [4]:
from dotwiz import DotWiz

CONSTANTS = DotWiz({
    "DATASETS": {
        "32BS_24MBS_5e-5LR_4E": {
            "TRAINING": "/kaggle/working/32bs-24mbs-5e-5lr-4e/training",
            "TESTING": "/kaggle/working/32bs-24mbs-5e-5lr-4e/testing",
        }
    },
    "MODELS": {
        "BERT": {
            "NAME": "bert-base-uncased",
            "TOKENIZER": {
                "MAX_LENGTH": 128
            },
            "BATCHING": {
                "SIZE": 32
            },
            "POOLER_OUTPUT_LENGTH": 768
        },
    },
    "PREPROCESSING": {
        "RANDOMNESS": {
            "SEED": 300
        },
        "DATASET_SPLITS": {
            "TRAINING": 0.8,
            "VALIDATION": 0.2,
        },
        "HYPERDATASET_SPLITS": {
            "TRAINING": 0.8,
            "VALIDATION": 0.2,
        },
        "HYPERDATASET_FRACTION": 0.4
    },
    "HYPERTUNING": {
        "OPTIMIZERS": {
            "SGD": {
                "LEARNING_RATE": {
                    "MIN_VALUE": 1e-2,
                    "MAX_VALUE": 1e-1,
                    "SAMPLING": "log"
                },
                "MOMENTUM": {
                    "MIN_VALUE": 1e-5,
                    "MAX_VALUE": 5e-5,
                    "SAMPLING": "log"
                }
            },
            "RMSPROP": {
                "LEARNING_RATE": {
                    "MIN_VALUE": 1e-5,
                    "MAX_VALUE": 5e-5,
                    "SAMPLING": "log"
                },
                "RHO": {
                    "MIN_VALUE": 1e-5,
                    "MAX_VALUE": 1e-1,
                    "SAMPLING": "log"
                },
                "MOMENTUM": {
                    "MIN_VALUE": 1e-5,
                    "MAX_VALUE": 5e-5,
                    "SAMPLING": "log"
                },
                "WEIGHT_DECAY": {
                    "MIN_VALUE": 1e-5,
                    "MAX_VALUE": 1e-1,
                    "SAMPLING": "log"
                }
            }
        },
        "LAYERS": {
            "LSTM": {
                "UNITS": {
                    "MIN_VALUE": 32,
                    "MAX_VALUE": 128,
                    "STEP": 32
                }
            },
            "DENSE": {
                "UNITS": {
                    "MIN_VALUE": 768,
                    "MAX_VALUE": 1024,
                    "STEP": 32
                },
            },
            "DROPOUT": {
                "RATE": {
                    "MIN_VALUE": 0.1,
                    "MAX_VALUE": 0.6,
                    "STEP": 0.1,
                },
            }
        },
        "ALGORITHMS": {
            "RANDOM_SEARCH": {
                "OBJECTIVE": "val_loss",
                "MAX_TRIALS": 50,
                "EPOCHS_PER_TRIAL": 5
            }
        },
        "PATHS": {
            "ROOT": "/kaggle/tmp/hypertuning",
            "HYPERBAND": {
                "SGD": "hyperband/sgd",
                "RMSPROP": "hyperband/rmsprop"
            },
            "RANDOM_SEARCH": {
                "SGD": "random_search/sgd",
                "RMSPROP": "random_search/rmsprop"
            },
        },
        "TRIALS_PER_EXECUTION": 5,
    },
    "TRAINING": {
        "CALLBACKS": {
            "MODEL_CHECKPOINT": {
                "MONITOR": "val_loss",
                "FILEPATH": "models/epoch-{epoch:02d}"
            }
        },
        "EPOCHS": 7
    },
    "INPUTS": {
        "FOCUS_MODEL": "32BS_24MBS_5e-5LR_4E"
    }
})

Para poder recrear (hasta cierto punto) cada uno de los resultados propuestos, preestableceremos una semilla. Es posible que existan diferencias entre una ejecución y otra ya que como trabajaremos a nivel de GPU, muchas de las operaciones en Tensorflow son procesadas de manera asíncrona, y muchos de los valores que tratamos acá requieren sumar flotantes que sí se ven afectados cuando cambian sus órdenes. Si quisiéramos habilitar un determinismo completo, usaríamos la instrucción `tensorflow.config.experimental.enable_op_determinism()`, pero veríamos una degradación en el desempeño de las instrucciones en varios órdenes de magnitud. Para mayores detalles, revisar la [documentación oficial](https://www.tensorflow.org/versions/r2.8/api_docs/python/tf/config/experimental/enable_op_determinism) de Tensorflow.

In [5]:
from tensorflow.keras.utils import set_random_seed

set_random_seed(CONSTANTS.PREPROCESSING.RANDOMNESS.SEED)

## Generación del conjunto de entrenamiento y validación

El conjunto de datos de **entrenamiento** PAN2012 tiene dos archivos de utilidad:

1. `pan12-sexual-predator-identification-training-corpus-predators-2012-05-01.txt`: Enlista todos los identificadores de los autores que (se sabe) son depredadores sexuales separados por saltos de línea.
1. `pan12-sexual-predator-identification-training-corpus-2012-05-01.xml`: Enlista tanto conversaciones normales como pervertidas en un formato de etiquetas. Cada conversación se encierra con `<conversation>` y cada mensaje por `<message>`.

En cuanto al directorio de pruebas, no existen muchas diferencias. Los archivos que nos interesan son los siguientes:

1. `pan12-sexual-predator-identification-groundtruth-problem1.txt`: Enlista todos los identificadores de los autores que (se sabe) son depredadores sexuales separados por saltos de línea.
1. `pan12-sexual-predator-identification-test-corpus-2012-05-17.xml`: Enlista tanto conversaciones normales como pervertidas en un formato de etiquetas. Cada conversación se encierra con `<conversation>` y cada mensaje por `<message>`.

La cantidad de conversaciones normales versus pervertidas está altamente desequilibrada, por lo que se hará un tratamiento básico. Los mensajes ya vienen agrupados desde la fase 1 y se cuentan con sus respectivos *embeddings*.

In [6]:
from datasets import load_from_disk, DatasetDict
from pandas import Series
from transformers import DefaultDataCollator
from tensorflow.keras.preprocessing.sequence import pad_sequences
from shutil import rmtree
from os.path import split


def pad_pooler_output(pooler_output):
    padded_pooler_output = pad_sequences(pooler_output,
                                         padding="post",
                                         dtype="float32")
    return {"padded_pooler_output": padded_pooler_output}


def balance_dataset(hf_dataset):
    conversation_series = Series(hf_dataset["conversation_label"])
    normal_conversation_series = conversation_series[conversation_series ==
                                                     False]
    perverted_conversation_series = conversation_series[conversation_series ==
                                                        True]
    normal_conversation_series = normal_conversation_series.sample(
        frac=len(perverted_conversation_series) /
        len(normal_conversation_series),
        random_state=CONSTANTS.PREPROCESSING.RANDOMNESS.SEED)

    hf_dataset = hf_dataset.select(
        perverted_conversation_series.index.to_list() +
        normal_conversation_series.index.to_list())

    return hf_dataset


hf_dataset = DatasetDict([
    ("train",
     load_from_disk(
         CONSTANTS.DATASETS[CONSTANTS.INPUTS.FOCUS_MODEL].TRAINING)),
    ("test",
     load_from_disk(CONSTANTS.DATASETS[CONSTANTS.INPUTS.FOCUS_MODEL].TESTING))
])
hf_dataset["train"] = balance_dataset(hf_dataset["train"])
hf_dataset["test"] = balance_dataset(hf_dataset["test"])

hf_dataset["train"] = hf_dataset["train"].train_test_split(
    train_size=CONSTANTS.PREPROCESSING.DATASET_SPLITS.TRAINING,
    shuffle=True,
    seed=CONSTANTS.PREPROCESSING.RANDOMNESS.SEED)

hf_training_dataset = hf_dataset["train"]["train"]
hf_validation_dataset = hf_dataset["train"]["test"]
hf_testing_dataset = hf_dataset["test"]

hf_hyperdataset = hf_training_dataset.train_test_split(
    train_size=CONSTANTS.PREPROCESSING.HYPERDATASET_FRACTION,
    shuffle=True,
    seed=CONSTANTS.PREPROCESSING.RANDOMNESS.SEED)["train"]
hf_hyperdataset = hf_hyperdataset.train_test_split(
    train_size=CONSTANTS.PREPROCESSING.HYPERDATASET_SPLITS.TRAINING,
    shuffle=True,
    seed=CONSTANTS.PREPROCESSING.RANDOMNESS.SEED)
hf_hypertraining_dataset = hf_hyperdataset["train"]
hf_hypervalidation_dataset = hf_hyperdataset["test"]

hf_training_dataset = hf_training_dataset.map(
    lambda example: pad_pooler_output(example["pooler_output"]),
    batched=True,
    batch_size=CONSTANTS.MODELS.BERT.BATCHING.SIZE)
hf_validation_dataset = hf_validation_dataset.map(
    lambda example: pad_pooler_output(example["pooler_output"]),
    batched=True,
    batch_size=CONSTANTS.MODELS.BERT.BATCHING.SIZE)
hf_testing_dataset = hf_testing_dataset.map(
    lambda example: pad_pooler_output(example["pooler_output"]),
    batched=True,
    batch_size=CONSTANTS.MODELS.BERT.BATCHING.SIZE)
hf_hypertraining_dataset = hf_hypertraining_dataset.map(
    lambda example: pad_pooler_output(example["pooler_output"]),
    batched=True,
    batch_size=CONSTANTS.MODELS.BERT.BATCHING.SIZE)
hf_hypervalidation_dataset = hf_hypervalidation_dataset.map(
    lambda example: pad_pooler_output(example["pooler_output"]),
    batched=True,
    batch_size=CONSTANTS.MODELS.BERT.BATCHING.SIZE)

default_data_collator = DefaultDataCollator(return_tensors="tf")
to_tf_dataset_kwargs = {
    "columns": ["padded_pooler_output"],
    "label_cols": ["conversation_label"],
    "batch_size": CONSTANTS.MODELS.BERT.BATCHING.SIZE,
    "collate_fn": default_data_collator,
    "shuffle": False
}

tf_training_dataset = hf_training_dataset.to_tf_dataset(**to_tf_dataset_kwargs)
tf_validation_dataset = hf_validation_dataset.to_tf_dataset(
    **to_tf_dataset_kwargs)
tf_testing_dataset = hf_testing_dataset.to_tf_dataset(**to_tf_dataset_kwargs)
tf_hypertraining_dataset = hf_hypertraining_dataset.to_tf_dataset(
    **to_tf_dataset_kwargs)
tf_hypervalidation_dataset = hf_hypervalidation_dataset.to_tf_dataset(
    **to_tf_dataset_kwargs)

rmtree(split(CONSTANTS.DATASETS[CONSTANTS.INPUTS.FOCUS_MODEL].TRAINING)[0])

  0%|          | 0/101 [00:00<?, ?ba/s]

  0%|          | 0/26 [00:00<?, ?ba/s]

  0%|          | 0/233 [00:00<?, ?ba/s]

  0%|          | 0/33 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

## Generar modelos dinámicamente en función de los hiperparámetros

El hiperafinador de Keras Tuner requiere que se defina una función que reciba a un objeto de hiperparámetros y se retorne un modelo en función de lo anterior.

In [7]:
from tensorflow.keras.layers import Dense, Input, Dropout, Masking, LSTM
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras import Model, Sequential


def model_builder(hp):
    loss = BinaryCrossentropy(from_logits=True)
    metrics = BinaryAccuracy()
    optimizer = SGD(learning_rate=hp.Float(
        "learning_rate",
        min_value=CONSTANTS.HYPERTUNING.OPTIMIZERS.SGD.LEARNING_RATE.MIN_VALUE,
        max_value=CONSTANTS.HYPERTUNING.OPTIMIZERS.SGD.LEARNING_RATE.MAX_VALUE,
        sampling=CONSTANTS.HYPERTUNING.OPTIMIZERS.SGD.LEARNING_RATE.SAMPLING))

    model = Sequential()
    model.add(
        Input(shape=(None, CONSTANTS.MODELS.BERT.POOLER_OUTPUT_LENGTH),
              dtype="float32"))
    model.add(Masking(mask_value=0, dtype="float32"))
    model.add(
        LSTM(units=hp.Int(
            "lstm_units",
            min_value=CONSTANTS.HYPERTUNING.LAYERS.LSTM.UNITS.MIN_VALUE,
            max_value=CONSTANTS.HYPERTUNING.LAYERS.LSTM.UNITS.MAX_VALUE,
            step=CONSTANTS.HYPERTUNING.LAYERS.LSTM.UNITS.STEP)))
    model.add(
        Dense(units=hp.Int(
            "dense_units",
            min_value=CONSTANTS.HYPERTUNING.LAYERS.DENSE.UNITS.MIN_VALUE,
            max_value=CONSTANTS.HYPERTUNING.LAYERS.DENSE.UNITS.MAX_VALUE,
            step=CONSTANTS.HYPERTUNING.LAYERS.DENSE.UNITS.STEP),
              activation="relu"))
    model.add(
        Dropout(rate=hp.Float(
            "dropout_rate",
            min_value=CONSTANTS.HYPERTUNING.LAYERS.DROPOUT.RATE.MIN_VALUE,
            max_value=CONSTANTS.HYPERTUNING.LAYERS.DROPOUT.RATE.MAX_VALUE,
            step=CONSTANTS.HYPERTUNING.LAYERS.DROPOUT.RATE.STEP)))
    model.add(Dense(1, activation=None, name="classifier"))
    model.compile(optimizer=optimizer, metrics=metrics, loss=loss)

    return model

## Limpieza de memoria a nivel de disco

De acuerdo a las [especificaciones técnicas](https://www.kaggle.com/docs/notebooks#technical-specifications) de los *notebooks* en Kaggle, la carpeta `/kaggle/working` tiene una capacidad máxima de 20 \[GB\]. Por ello, crearemos la carpeta temporal `/kaggle/tmp` que presentará dos ventajas:

1. Tendrá una capacidad en disco cercana al triple de lo que hay en `/kaggle/working`.
1. Cualquier archivo generado en `/kaggle/tmp` será borrado una vez se cierre la sesión.

Solo se utilizará para guardar los *trials* de la tapa de hiperafinación.

In [8]:
from os import makedirs

makedirs(CONSTANTS.HYPERTUNING.PATHS.ROOT)

## Etapa de hiperafinación

Realizaremos la búsqueda de los hiperparámetros a través del método de búsqueda aleatoria pues permite ahondar mucho más el espacio de búsqueda. Se construirá un modelo a partir de los mejores hiperparámetros.

In [9]:
from os.path import join
from shutil import rmtree
from keras_tuner import RandomSearch

tuner = RandomSearch(
    hypermodel=model_builder,
    objective=CONSTANTS.HYPERTUNING.ALGORITHMS.RANDOM_SEARCH.OBJECTIVE,
    max_trials=CONSTANTS.HYPERTUNING.ALGORITHMS.RANDOM_SEARCH.MAX_TRIALS,
    seed=CONSTANTS.PREPROCESSING.RANDOMNESS.SEED,
    directory=CONSTANTS.HYPERTUNING.PATHS.ROOT,
    project_name=CONSTANTS.HYPERTUNING.PATHS.RANDOM_SEARCH.SGD)

tuner.search(
    tf_hypertraining_dataset,
    validation_data=tf_hypervalidation_dataset,
    epochs=CONSTANTS.HYPERTUNING.ALGORITHMS.RANDOM_SEARCH.EPOCHS_PER_TRIAL)

best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hyperparameters)

print("\nhyperparameters:", best_hyperparameters.values)

Trial 50 Complete [00h 00m 21s]
val_loss: 0.10454408079385757

Best val_loss So Far: 0.0793808102607727
Total elapsed time: 00h 16m 13s

hyperparameters: {'learning_rate': 0.09901356518045402, 'lstm_units': 64, 'dense_units': 768, 'dropout_rate': 0.5}


## Etapa de entrenamiento

Se definirá un único *callback* llamado `ModelCheckpoint` que permitirá guardar los modelos obtenidos en cada *epoch*.

In [10]:
from tensorflow.keras.callbacks import ModelCheckpoint

history = model.fit(
    tf_training_dataset,
    validation_data=tf_validation_dataset,
    callbacks=[
        ModelCheckpoint(
            CONSTANTS.TRAINING.CALLBACKS.MODEL_CHECKPOINT.FILEPATH,
            monitor=CONSTANTS.TRAINING.CALLBACKS.MODEL_CHECKPOINT.MONITOR,
            save_best_only=False,
            save_weights_only=False)
    ],
    epochs=CONSTANTS.TRAINING.EPOCHS,
    verbose=1)

Epoch 1/7
101/101 [==============================] - 20s 157ms/step - loss: 0.2505 - binary_accuracy: 0.8750 - val_loss: 0.1381 - val_binary_accuracy: 0.9392
Epoch 2/7
101/101 [==============================] - 14s 141ms/step - loss: 0.1278 - binary_accuracy: 0.9479 - val_loss: 0.1222 - val_binary_accuracy: 0.9491
Epoch 3/7
101/101 [==============================] - 14s 139ms/step - loss: 0.1096 - binary_accuracy: 0.9572 - val_loss: 0.1165 - val_binary_accuracy: 0.9491
Epoch 4/7
101/101 [==============================] - 15s 146ms/step - loss: 0.1055 - binary_accuracy: 0.9575 - val_loss: 0.1120 - val_binary_accuracy: 0.9578
Epoch 5/7
101/101 [==============================] - 14s 142ms/step - loss: 0.1000 - binary_accuracy: 0.9631 - val_loss: 0.1088 - val_binary_accuracy: 0.9615
Epoch 6/7
101/101 [==============================] - 15s 146ms/step - loss: 0.0965 - binary_accuracy: 0.9662 - val_loss: 0.1105 - val_binary_accuracy: 0.9603
Epoch 7/7
101/101 [==============================] -

## Estimación de métricas en base al conjunto de pruebas

Dado que los conjuntos que trabajamos son instancias de la clase padre `tensorflow.data.Dataset`, no podemos acceder directamente a las etiquetas en un formato *slicing* (o similar), por lo que deberemos recorrer *batch* por *batch* y concatenar los resultados en un vector. 

In [11]:
from numpy import array, concatenate


def get_tf_dataset_labels(tf_dataset):
    concatenated_labels = array([], dtype="int32")

    for _, labels in tf_dataset:
        concatenated_labels = concatenate((concatenated_labels, labels))

    return concatenated_labels

true_labels = get_tf_dataset_labels(tf_testing_dataset)

Para medir la capacidad de generalización del modelo recién entrenado, definiremos la función `mesaure_model_metrics`, la cual retornará cinco valores (en orden):

1. Exactitud o *binary accuracy*.
1. Pérdida o *binary crossentropy*.
1. Recuperación o *recall*.
1. Precisión o *precision*.
1. Valor-F o *F-score*.

In [12]:
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.losses import BinaryCrossentropy


def measure_model_metrics(true_labels, predicted_labels):
    binary_accuracy = BinaryAccuracy()
    binary_accuracy.update_state(true_labels, predicted_labels)

    binary_crossentropy = BinaryCrossentropy(from_logits=False)(
        true_labels, predicted_labels)

    recall = Recall()
    recall.update_state(true_labels, predicted_labels)

    precision = Precision()
    precision.update_state(true_labels, predicted_labels)

    f_score = 2 * (precision.result().numpy() * recall.result().numpy()) / (
        precision.result().numpy() + recall.result().numpy())

    return binary_accuracy.result().numpy(), binary_crossentropy.numpy(
    ), recall.result().numpy(), precision.result().numpy(), f_score

Finalmente, obtendremos las métricas de desempeño del modelo durante cada *epoch*.

In [13]:
from tensorflow.keras.saving import load_model
from numpy import array
from tensorflow import sigmoid
from sklearn.metrics import confusion_matrix as measure_confusion_matrix

THRESHOLD = 0.5

for epoch in range(CONSTANTS.TRAINING.EPOCHS):
    model = load_model("models/epoch-{epoch:02d}".format(epoch=epoch + 1))
    predicted_labels = sigmoid(model.predict(tf_testing_dataset,
                                             verbose=1)).numpy().flatten()
    accuracy, loss, recall, precision, f_score = measure_model_metrics(
        true_labels, predicted_labels)

    confusion_matrix = measure_confusion_matrix(true_labels,
                                                predicted_labels > THRESHOLD)

    print("\n========\nepoch {epoch:02d}\n========\n".format(epoch=epoch + 1))
    print("metrics\n=======\n")
    print(f"- accuracy: {accuracy}")
    print(f"- loss: {loss}")
    print(f"- recall: {recall}")
    print(f"- precision: {precision}")
    print(f"- f-score: {f_score}")
    print("\nconfusion matrix\n================\n")
    print(confusion_matrix, "\n\n")

233/233 [==============================] - 10s 35ms/step

epoch 01

metrics

- accuracy: 0.9264034628868103
- loss: 0.17338338494300842
- recall: 0.8984689712524414
- precision: 0.9516358375549316
- f-score: 0.9242884768390116

confusion matrix

[[3553  170]
 [ 378 3345]] 


233/233 [==============================] - 9s 35ms/step

epoch 02

metrics

- accuracy: 0.9429223537445068
- loss: 0.16371969878673553
- recall: 0.9323126673698425
- precision: 0.952524721622467
- f-score: 0.9423103121706046

confusion matrix

[[3550  173]
 [ 252 3471]] 


233/233 [==============================] - 9s 35ms/step

epoch 03

metrics

- accuracy: 0.940907895565033
- loss: 0.17054566740989685
- recall: 0.925597608089447
- precision: 0.9548351168632507
- f-score: 0.9399890821811523

confusion matrix

[[3560  163]
 [ 277 3446]] 


233/233 [==============================] - 10s 36ms/step

epoch 04

metrics

- accuracy: 0.9429223537445068
- loss: 0.16825102269649506
- recall: 0.9320440292358398
- precision:

## Guardar resultados

Con el afán de poder recrear los resultados, se persisten los conjuntos de datos generados.

In [14]:
hf_training_dataset.save_to_disk("datasets/training")
hf_validation_dataset.save_to_disk("datasets/validation")
hf_testing_dataset.save_to_disk("datasets/testing")
hf_hypertraining_dataset.save_to_disk("datasets/hypertraining")
hf_hypervalidation_dataset.save_to_disk("datasets/hypervalidation")